In [ ]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=fa003600dbae0ba2c0a0e53bb2d16cb5cb8a9604753510999a14a3b12030f0b9
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import requests
import regex as re
import wget
import os

In [ ]:
from re import X
import numpy as np
import torch
from torch.utils.data import Dataset
import torch.nn as nn
from argparse import ArgumentParser, Namespace
from pathlib import Path
from typing import Dict
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from utils import *
import xarray as xr
import datetime
import pandas as pd

In [ ]:

sd = datetime.datetime(2023, 5, 12).strftime("%Y%m%d")
ed = datetime.datetime(2023,5,12).strftime("%Y%m%d")

d1=datetime.date(2023,5,12)
d2=datetime.date(2023,5,12)

delta=(d2-d1).days

In [ ]:
ls=pd.date_range(sd,ed)

In [ ]:
dates=[dt.strftime("%Y%m%d") for dt in ls]

In [ ]:
years={date[:4] for date in dates}
months={date[4:6] for date in dates}

In [ ]:
txt=""
for year in years:
    for month in months:
        base='https://www.ngdc.noaa.gov/dscovr/data/'+year+'/'+month+'/'
        txt+=requests.get(base).text

In [ ]:
urllist=[]

In [ ]:
for date in dates:
    base='https://www.ngdc.noaa.gov/dscovr/data/'+date[:4]+'/'+date[4:6]+'/'
    x=re.search(str("oe_m1m_dscovr_s"+date+"000000_"), txt).start()
    url=base+txt[x: x+71]
    print(date,": ",url)
    urllist.append(url[46:])
    wget.download(url)

20230512 :  https://www.ngdc.noaa.gov/dscovr/data/2023/05/oe_m1m_dscovr_s20230512000000_e20230512235959_p20230513021827_pub.nc.gz


In [ ]:
l=len(urllist)

In [ ]:
magdata=[]

In [ ]:
for i in range(0,l):
  ds=xr.open_dataset('/content/'+urllist[i])
  df=ds.to_dataframe()
  actualdf=df.dropna()
  df_len=len(actualdf)
  for j in range(0,df_len,180):
   bx_gse_hour=np.nanmean(np.array(actualdf['bx_gse'][j:j+180]))
   by_gse_hour=np.nanmean(np.array(actualdf['by_gse'][j:j+180]))
   bz_gse_hour=np.nanmean(np.array(actualdf['bz_gse'][j:j+180]))
   b_gse=((bx_gse_hour)**2+(by_gse_hour)**2+(bz_gse_hour)**2)**0.5
   if bx_gse_hour+by_gse_hour+bz_gse_hour<0:
    magdata.append([-b_gse,-b_gse,-b_gse])
   else:
    magdata.append([b_gse,b_gse,b_gse])

In [ ]:
magdata_numpy=np.array(magdata)

In [ ]:
magdata_torch= torch.from_numpy(magdata_numpy)

In [ ]:
ml1=magdata_torch.unsqueeze(-1)

In [ ]:
kpindexdata=requests.get('https://services.swpc.noaa.gov/text/3-day-geomag-forecast.txt').text

In [ ]:
lkp=kpindexdata[470:].strip().split()
forecastdata=[]
lenkp=len(lkp)
for i in range(7,lenkp,4):
    forecastdata.append([float(lkp[i]),float(lkp[i+1]),float(lkp[i+2])])

In [ ]:
fnumpy=np.array(forecastdata)

In [ ]:
forecastdata_torch= torch.from_numpy(fnumpy)

In [ ]:
ft=forecastdata_torch.unsqueeze(-1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        output_dim: int,
        num_layers: int) -> None:
        super(Seq2Seq, self).__init__()

        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.dnn = nn.Sequential(
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, seq: torch.Tensor) -> torch.Tensor:
        output, hn = self.rnn(seq)
        output = self.dnn(output)
        return output

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class DSCOVRKP():
	def __init__(self):
		self.x_prime = torch.tensor([]).float()
		self.forecast = torch.tensor([]).float()
		self.x_prime = torch.cat([self.x_prime, ml1.float()], dim=0)
		self.forecast = torch.cat([self.forecast, ft.float()], dim=0)
		
		print("total x_prime shape:", self.x_prime.shape)
		print("total forecast shape:", self.forecast.shape)


	def __len__(self) -> int:
		return self.forecast.shape[0]

	def __getitem__(self, index):
		return self.x_prime[index],self.forecast[index]

In [ ]:
  d=DSCOVRKP()
  data_len = len(d)
  train_ratio,test_ratio = 0.7,0.3
  train_len= int(data_len*train_ratio)
  test_len = data_len - train_len
  train_dataset,test_dataset = random_split(d, [train_len,test_len])
  print(f"train,test dataset len: {len(train_dataset)}, {len(test_dataset)}")
  train_loader = DataLoader(train_dataset, 8, shuffle=True)
  test_loader = DataLoader(test_dataset, 8, shuffle=False)
  model = Seq2Seq(1,3,1,3)
  print(model)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.8)
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, min_lr=5e-5)
  loss_train=0
  prev_train=100
  while abs(loss_train-prev_train)>0.001:
    model.train()
    prev_train=loss_train
    loss_train, acc_train, iterations = 0, 0, 0
    for x_prime, forecast in train_loader:
      outputs = model(x_prime)
      criterion = torch.nn.MSELoss()
      newdst=forecast
      loss = criterion(outputs, newdst)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      iterations += 1
      loss_train += loss.item()
    loss_train /= iterations
    print(f"train_loss: {loss_train:.4f}")
    torch.save(model.state_dict(), os.path.join('/content/drive/MyDrive', "proton_to_forecast1.pt"))
		



total x_prime shape: torch.Size([8, 3, 1])
total forecast shape: torch.Size([8, 3, 1])
train,test dataset len: 5, 3
Seq2Seq(
  (rnn): GRU(1, 3, num_layers=3, batch_first=True)
  (dnn): Sequential(
    (0): Linear(in_features=3, out_features=1, bias=True)
  )
)
train_loss: 3.3447
train_loss: 0.5546
train_loss: 0.5381
train_loss: 2.1819
train_loss: 0.3538
train_loss: 1.6235
train_loss: 0.8915
train_loss: 0.2037
train_loss: 0.8103
train_loss: 0.7890
train_loss: 0.3325
train_loss: 0.4520
train_loss: 0.7326
train_loss: 0.5528
train_loss: 0.2877
train_loss: 0.3593
train_loss: 0.4702
train_loss: 0.2961
train_loss: 0.1819
train_loss: 0.3073
train_loss: 0.3781
train_loss: 0.2717
train_loss: 0.2102
train_loss: 0.2909
train_loss: 0.3332
train_loss: 0.2544
train_loss: 0.2031
train_loss: 0.2497
train_loss: 0.2748
train_loss: 0.2196
train_loss: 0.1836
train_loss: 0.2183
train_loss: 0.2323
train_loss: 0.1857
train_loss: 0.1704
train_loss: 0.2005
train_loss: 0.1928
train_loss: 0.1648
train_loss: 0.182

In [ ]:

  model = Seq2Seq(1,3,1,3)
  model.load_state_dict(torch.load(os.path.join('/content/drive/MyDrive', "proton_to_forecast1.pt")))
  model.eval()
  with torch.no_grad():
    loss_test, acc_test,iterations = 0, 0, 0 
    for x_prime, forecast in test_loader:
      outputs = model(x_prime)
      loss = criterion(outputs,forecast)
      iterations += 1
      loss_test += loss.item()
      for i in range(outputs.shape[0]):
       for j in range(outputs.shape[1]):
        for k in range(outputs.shape[2]):
          if loss_test<1.1 and abs(outputs[i][j][k]-forecast[i][j][k])<=loss_test:
            acc_test+=1
          elif abs(outputs[i][j][k]-forecast[i][j][k])<1.0:
            acc_test+=1
    loss_test /= iterations
    acc_test /= (outputs.shape[0])*(outputs.shape[1])*(outputs.shape[2])
  print(f"test_acc: {acc_test*100:.4f},test_loss: {loss_test:.4f}")

test_acc: 88.8889,test_loss: 0.3564


In [ ]:
outputs

tensor([[[3.0223],
         [2.3035],
         [1.7580]],

        [[2.4949],
         [1.8840],
         [1.1419]],

        [[2.4949],
         [1.8840],
         [1.1419]]])